In [13]:
# !wget https://huggingface.co/datasets/malaysia-ai/Malaysian-NSFW/resolve/main/malaysian-dataset-sfw-dedup-v2.jsonl
# !wget https://raw.githubusercontent.com/6/stopwords-json/master/dist/en.json

In [1]:
import malaya
import json
import os
from tqdm import tqdm
from collections import defaultdict
from sklearn.decomposition import LatentDirichletAllocation
from malaya.text.vectorizer import TfidfVectorizer

/root/app/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/app/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/root/app/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
labels = defaultdict(list)
with open('malaysian-dataset-sfw-dedup-v2.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        labels[l['label']].append(l['text'])
len(labels)

10

In [14]:
with open('en.json') as fopen:
    en = json.load(fopen)
    
stopwords = malaya.text.function.get_stopwords()
stopwords = stopwords + en + ['lor', 'pm', 'long', 'jer', 'time', 'feel', 'liao', 'wow', 'https', 'http', 've', 'ko', 'kena', 'post', 'ni', 'tu', 'don', 'je', 'jeh', 'la', 'tau', 'haha', 'hahaha', 'hahahaha']
stopwords += ['for me', 'to be', 'in the', 'me to', 'for me to']

In [16]:
for k in tqdm(labels.keys()):
    filename = f"{k.replace(' ', '-')}.json"
    if os.path.exists(filename):
        continue
    vectorizer = TfidfVectorizer(
        max_df = 0.95,
        min_df = 2,
        ngram_range = (1, 1),
        stop_words = stopwords
    )
    lda = malaya.topic_model.decomposition.fit(
        labels[k],
        LatentDirichletAllocation,
        vectorizer = vectorizer,
        n_topics = 1,
    )
    unigram = lda.top_topics(1, top_n = 10, return_df = False)
    
    vectorizer = TfidfVectorizer(
        max_df = 0.95,
        min_df = 2,
        ngram_range = (2, 2),
        stop_words = stopwords
    )
    lda = malaya.topic_model.decomposition.fit(
        labels[k],
        LatentDirichletAllocation,
        vectorizer = vectorizer,
        n_topics = 1,
    )
    bigram = lda.top_topics(1, top_n = 10, return_df = False)
    
    with open(filename, 'w') as fopen:
        json.dump([unigram, bigram], fopen)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [01:55<00:00, 11.54s/it]
